In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter

from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.cit import kci

from epc import epc

In [4]:
def gdata(n):
    """
    Generate data based on a causal graph

    Args:
        n (int): Number of samples.

    Returns:
        np.ndarray
    """
    def random_nonlinear_function(x, function_type):
        """Applies a nonlinear transformation based on the specified type."""
        if function_type == "linear":
            return x
        elif function_type == "cubic":
            return x ** 3
        elif function_type == "tanh":
            return np.tanh(x)
        else:
            raise ValueError("Unsupported function type")


    F_type_B = np.random.choice(["linear", "cubic", "tanh"])
    F_type_C = np.random.choice(["linear", "cubic", "tanh"])
    F_type_D = np.random.choice(["linear", "cubic", "tanh"])
    F_type_E = np.random.choice(["linear", "cubic", "tanh"])

    E_B = np.random.normal(0, 1, n)
    E_C = np.random.normal(0, 1, n)
    E_D = np.random.normal(0, 1, n)
    E_E = np.random.normal(0, 1, n)

    A = np.random.normal(0, 3, n)
    B = random_nonlinear_function(A + E_B, F_type_B)
    C = random_nonlinear_function(B + E_C + A/3, F_type_C)
    D = random_nonlinear_function(C + E_D, F_type_D)
    E = random_nonlinear_function(D + E_E + C/3, F_type_E)

    A = (A - np.mean(A)) / np.std(A)
    B = (B - np.mean(B)) / np.std(B)
    C = (C - np.mean(C)) / np.std(C)
    D = (D - np.mean(D)) / np.std(D)
    E = (E - np.mean(E)) / np.std(E)

    data = np.array([A, B, C, D, E]).T

    return data


truSk = np.array([[ 0,  1,  1,  0,  0],
                  [ 1,  0,  1,  0,  0],
                  [ 1,  1,  0,  1,  1],
                  [ 0,  0,  1,  0,  1],
                  [ 0,  0,  1,  1,  0]])

In [7]:
t = 100
n = 800
err = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    cg = pc(data, 0.01, kci, show_progress=False)
    sk = np.abs(cg.G.graph)
    if not np.all(np.abs(sk) == truSk):
        err += 1
err/t

Processing: 100%|██████████| 100/100 [48:06<00:00, 28.87s/it]


0.8

In [5]:
t = 100
n = 800
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "Gamma", 8, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing:  53%|█████▎    | 53/100 [13:35<13:27, 17.18s/it]/Users/gzk/Library/Mobile Documents/iCloud~md~obsidian/Documents/CD/_Project/Ensemble KCIT/code/gammaKCIT.py:239: RuntimeWarning: invalid value encountered in scalar divide
  k_appr = mean_appr ** 2 / var_appr
/Users/gzk/Library/Mobile Documents/iCloud~md~obsidian/Documents/CD/_Project/Ensemble KCIT/code/gammaKCIT.py:240: RuntimeWarning: invalid value encountered in scalar divide
  theta_appr = var_appr / mean_appr
/Users/gzk/Library/Mobile Documents/iCloud~md~obsidian/Documents/CD/_Project/Ensemble KCIT/code/gammaKCIT.py:601: RuntimeWarning: invalid value encountered in scalar divide
  k_appr = mean_appr ** 2 / var_appr
/Users/gzk/Library/Mobile Documents/iCloud~md~obsidian/Documents/CD/_Project/Ensemble KCIT/code/gammaKCIT.py:602: RuntimeWarning: invalid value encountered in scalar divide
  theta_appr = var_appr / mean_appr
Processing: 100%|██████████| 100/100 [26:01<00:00, 15.62s/it]


0.68

In [9]:
t = 100
n = 800
err_e = 0

for i in tqdm(range(t), desc="Processing"):
    np.random.seed(i)
    data = gdata(n)
    ecg = epc(data, 0.01, "ACAT", 4, show_progress=False)
    esk =np.abs(ecg.G.graph)
    if not np.all(np.abs(esk) == truSk):
        err_e += 1
err_e/t

Processing: 100%|██████████| 100/100 [21:15<00:00, 12.76s/it]


0.96